# ***Vibrio natriegens:*** **Useful Info from iLC858 model**

In [20]:
import cobra
import torch
from cobra.io import read_sbml_model
model = read_sbml_model('C:/Users/Ainara/Documents/GitHub/Special-Course/models/iLC858.sbml')
media=model.medium

### **Default Media: Components**

In order to have an idea of the nutrients that are essential for growing, we printed the components of the default media from the model. The exchange reactions ("EX_") from the extracellular compartment ('_e') are shown as the nutrients that are in the default media. The reactions ID were translated to easy readable names.

In [2]:
# Map exchange reactions to human-readable metabolite names
for reaction_id, flux in media.items():
    reaction = model.reactions.get_by_id(reaction_id)
    metabolite = list(reaction.metabolites.keys())[0]  # Get the first metabolite in the reaction
    if "_e" in metabolite.id: 
        print(f"{reaction_id}: {metabolite.name} ({flux} mmol/gDW/h)")

EX_cpd00009_e: Phosphate_e (1000.0 mmol/gDW/h)
EX_cpd00011_e: CO2_e (1000.0 mmol/gDW/h)
EX_cpd00012_e: PPi_e (1000.0 mmol/gDW/h)
EX_cpd00013_e: NH3_e (1000.0 mmol/gDW/h)
EX_cpd00030_e: Mn2+_e (1000.0 mmol/gDW/h)
EX_cpd00034_e: Zn2+_e (1000.0 mmol/gDW/h)
EX_cpd00048_e: Sulfate_e (1000.0 mmol/gDW/h)
EX_cpd00058_e: Cu2+_e (1000.0 mmol/gDW/h)
EX_cpd00063_e: Ca2+_e (1000.0 mmol/gDW/h)
EX_cpd00067_e: H+_e (1000.0 mmol/gDW/h)
EX_cpd00099_e: Cl-_e (1000.0 mmol/gDW/h)
EX_cpd00149_e: Co2+_e (1000.0 mmol/gDW/h)
EX_cpd00205_e: K+_e (1000.0 mmol/gDW/h)
EX_cpd00254_e: Mg_e (1000.0 mmol/gDW/h)
EX_cpd00531_e: Hg2+_e (1000.0 mmol/gDW/h)
EX_cpd00971_e: Na+_e (1000.0 mmol/gDW/h)
EX_cpd04097_e: Pb_e (1000.0 mmol/gDW/h)
EX_cpd10516_e: Fe+3_e (1000.0 mmol/gDW/h)


### **Default Max. Growth**
Exchange reactions were set to high values to test the theoretical maximum growth.

In [3]:
with model:
    for exchange in model.medium.keys():
        model.medium[exchange] = 1000.0
    print(f"Max theoretical growth:", model.optimize().objective_value)

Max theoretical growth: 2.2528487841273113


### **Possible Limiting Nutrients**

**Shadow prices** in FBA represents how much the objective function would improve if you relax the constraint on a particular nutrient. Negative shadow prices indicates that the metabolite is limiting growth. If more of this metabolite is available, the growth rate will increase.

Shadow prices were caluculated for all nutrients. Nutrients with a negative shadow prices are likely to be limiting. In that way we could evaluate which nutrients **must** be in our media in order to make bacteria grows.

In [4]:
solution = model.optimize()
shadow_prices = solution.reduced_costs
shadow_prices = solution.reduced_costs

# Create a list of limiting nutrients (negative shadow prices)
limiting_nutrients = []
for rxn_id, price in shadow_prices.items():
    if "EX_" in rxn_id and price < 0:  # Only consider exchange reactions with negative values
        reaction = model.reactions.get_by_id(rxn_id)
        metabolite = list(reaction.metabolites.keys())[0]  # Get the associated metabolite
        limiting_nutrients.append((metabolite.name, price))

# Sort by shadow price (most negative first)
limiting_nutrients.sort(key=lambda x: x[1])  # Sort by price (ascending, more negative first)

# Print the ranked list of limiting nutrients
print("**Ranked Limiting Nutrients**:")
for rank, (metabolite_name, price) in enumerate(limiting_nutrients, start=1):
    print(f"{rank}. {metabolite_name}: {price:.4f}")

**Ranked Limiting Nutrients**:
1. Heme_e: -1.4746
2. Thiamin_e: -1.3500
3. ocdca_e: -0.8740
4. Palmitate_e: -0.7629
5. Myristic acid_e: -0.6712
6. L-Tryptophan_e: -0.4599
7. Sucrose_e: -0.4153
8. TRHL_e: -0.4153
9. Maltose_e: -0.4056
10. Spermidine_e: -0.3609
11. L-Tyrosine_e: -0.3497
12. PAN_e: -0.3404
13. N-Acetyl-D-glucosamine_e: -0.2728
14. L-Methionine_e: -0.2716
15. L-Lysine_e: -0.2511
16. L-Histidine_e: -0.2382
17. L-Arginine_e: -0.2257
18. D-Mannitol_e: -0.2233
19. Putrescine_e: -0.2185
20. GLUM_e: -0.2173
21. L-Rhamnose_e: -0.2113
22. D-Fructose_e: -0.2076
23. D-Glucose_e: -0.2076
24. HYXN_e: -0.1984
25. Galactose_e: -0.1980
26. Ornithine_e: -0.1823
27. L-Proline_e: -0.1775
28. L-Arabinose_e: -0.1706
29. L-Glutamine_e: -0.1533
30. Citrate_e: -0.1533
31. L-Glutamate_e: -0.1485
32. XAN_e: -0.1473
33. Cytosine_e: -0.1453
34. Uracil_e: -0.1429
35. Glycerol-3-phosphate_e: -0.1412
36. L-Asparagine_e: -0.1328
37. Glycerol_e: -0.1195
38. Succinate_e: -0.1099
39. L-Aspartate_e: -0.1038

### **Expected growth for our Baseline Media**

We simulated our media and updated the default media to calculate the expected growth we would obtain in the lab. 

In [11]:
new_medium = {}
baseline_medium = {
    'EX_cpd00099_e': 64.8, #clhoride
    'EX_cpd00971_e': 91.6, #sodium
    'EX_cpd00048_e': 121.0, #sulfate
    'EX_cpd00205_e': 136.3, #potassium
    'EX_cpd00063_e': 0.2, #calcium
    'EX_cpd00149_e': 0.1, #carbonate
    'EX_cpd00254_e': 121.6, #magnesium
    'EX_cpd00029_e': 82.0, #acetate
    'EX_cpd00013_e': 53.5, #ammonium
    'EX_cpd00012_e': 136.1, #H2PO4
    'EX_cpd00305_e': 0.014, #thiamine/vit B1
    'EX_cpd00635_e': 0.0007378, #vit B12
    'EX_cpd00028_e': 0.010790591 #iron
}

for nutrient, concentration in baseline_medium.items():
    if nutrient in exchange_reactions:
        new_medium[nutrient] = concentration
    else:
        print(f"Warning: {nutrient} not found in model exchange reactions.")

# Update the model's medium
model.medium = new_medium
solution = model.optimize()  # Optimize the model
max_theoretical_growth = solution.objective_value  # Extract max theoretical grow
print(f'Expected growth for our Baseline Media:', max_theoretical_growth)

Expected growth for our Baseline Media: 1.0232461816187762e-13


As our strain got a growth close to 0 with the stablished media, we tried to add one per one each nutrient from the default media to our media, in order to find the nutrient that is missing and causing the lack of growth. 

In [17]:
# Get all exchange reactions available in the model
exchange_reactions = [rxn.id for rxn in model.exchanges]

# Initialize a new medium with only the desired components (if they exist)
new_medium = {}
# Define desired nutrients and their concentrations
baseline_medium = {
    'EX_cpd00099_e': 64.8, #clhoride
    'EX_cpd00971_e': 91.6, #sodium
    'EX_cpd00048_e': 121.0, #sulfate
    'EX_cpd00205_e': 136.3, #potassium
    'EX_cpd00063_e': 0.2, #calcium
    'EX_cpd00149_e': 0.1, #carbonate
    'EX_cpd00254_e': 121.6, #magnesium
    'EX_cpd00029_e': 82.0, #acetate
    'EX_cpd00013_e': 53.5, #ammonium
    'EX_cpd00012_e': 136.1, #H2PO4
    'EX_cpd00305_e': 0.014, #thiamine/vit B1
    'EX_cpd00635_e': 0.0007378, #vit B12
    'EX_cpd00028_e': 0.010790591 #iron
}

# Only add nutrients that exist as exchange reactions in the model
for nutrient, concentration in baseline_medium.items():
    if nutrient in exchange_reactions:
        new_medium[nutrient] = concentration
    else:
        print(f"Warning: {nutrient} not found in model exchange reactions.")

# Update the model's medium
model.medium = new_medium
print(model.medium, f"Length baseline medium:", len(baseline_medium))

solution = model.optimize()
max_theoretical_growth = solution.objective_value
print(f"Expected growth for baseline media:", max_theoretical_growth)

essential_components = {
    'EX_cpd00009_e': 1000.0,  # Phosphate
    'EX_cpd00011_e': 1000.0,  # CO2
    'EX_cpd00012_e': 1000.0,  # PPi
    'EX_cpd00013_e': 1000.0,  # NH3
    'EX_cpd00030_e': 1000.0,  # Mn2+
    'EX_cpd00034_e': 1000.0,  # Zn2+
    'EX_cpd00048_e': 1000.0,  # Sulfate
    'EX_cpd00058_e': 1000.0,  # Cu2+
    'EX_cpd00063_e': 1000.0,  # Ca2+
    'EX_cpd00067_e': 1000.0,  # H+
    'EX_cpd00099_e': 1000.0,  # Cl-
    'EX_cpd00149_e': 1000.0,  # Co2+
    'EX_cpd00205_e': 1000.0,  # K+
    'EX_cpd00254_e': 1000.0,  # Mg2+
    'EX_cpd00531_e': 1000.0,  # Hg2+
    'EX_cpd00971_e': 1000.0,  # Na+
    'EX_cpd04097_e': 1000.0,  # Pb
    'EX_cpd10516_e': 1000.0   # Fe+3
}

# Start with my baseline medium
base_medium = baseline_medium.copy()

for component, concentration in essential_components.items():
    print(f"Testing {component}:")

    # Add one extra component on top of your custom medium
    test_medium = base_medium.copy()
    test_medium[component] = concentration
    
    # Apply the new medium
    model.medium = test_medium
    
    solution = model.optimize()
    print(f"{component} --> Growth: {solution.objective_value}")
    print(f"length test medium:", len(test_medium))

    # Stop the loop if growth is detected
    if solution.objective_value > max_theoretical_growth:
        print(f"BINGO! {component} is the missing piece!")
        break

{'EX_cpd00012_e': 136.1, 'EX_cpd00013_e': 53.5, 'EX_cpd00028_e': 0.010790591, 'EX_cpd00048_e': 121.0, 'EX_cpd00063_e': 0.2, 'EX_cpd00099_e': 64.8, 'EX_cpd00149_e': 0.1, 'EX_cpd00205_e': 136.3, 'EX_cpd00254_e': 121.6, 'EX_cpd00305_e': 0.014, 'EX_cpd00635_e': 0.0007378, 'EX_cpd00971_e': 91.6, 'EX_cpd00029_e': 82.0} Length baseline medium: 13
Expected growth for baseline media: 1.3029933063366596e-13
Testing EX_cpd00009_e:
EX_cpd00009_e --> Growth: -1.4656252838958794e-16
length test medium: 14
Testing EX_cpd00011_e:
EX_cpd00011_e --> Growth: 0.0
length test medium: 14
Testing EX_cpd00012_e:
EX_cpd00012_e --> Growth: 0.0
length test medium: 13
Testing EX_cpd00013_e:
EX_cpd00013_e --> Growth: 0.0
length test medium: 13
Testing EX_cpd00030_e:
EX_cpd00030_e --> Growth: 0.0
length test medium: 14
Testing EX_cpd00034_e:
EX_cpd00034_e --> Growth: 0.0
length test medium: 14
Testing EX_cpd00048_e:
EX_cpd00048_e --> Growth: 0.0
length test medium: 13
Testing EX_cpd00058_e:
EX_cpd00058_e --> Growth

As most nutrients have synergy, we tried as well to add them in couples and triplets. 

In [18]:
import itertools
couples = list(itertools.combinations(essential_components, 2))

for triplet in couples:
    medium = {c: 1000.0 for c in triplet}
    model.medium = medium
    solution = model.optimize()
    print(f"{triplet} → Growth: {solution.objective_value}")

('EX_cpd00009_e', 'EX_cpd00011_e') → Growth: 2.901984972733107e-14
('EX_cpd00009_e', 'EX_cpd00012_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00013_e') → Growth: -4.2705725209512465e-14
('EX_cpd00009_e', 'EX_cpd00030_e') → Growth: -8.85781508224639e-15
('EX_cpd00009_e', 'EX_cpd00034_e') → Growth: -8.85781508224639e-15
('EX_cpd00009_e', 'EX_cpd00048_e') → Growth: -1.88512474827295e-14
('EX_cpd00009_e', 'EX_cpd00058_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00063_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00067_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00099_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00149_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00205_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00254_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00531_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00971_e') → Growth: 1.1780

In [19]:
triplets = list(itertools.combinations(essential_components, 3))

for triplet in triplets:
    medium = {c: 1000.0 for c in triplet}
    model.medium = medium
    solution = model.optimize()
    print(f"{triplet} → Growth: {solution.objective_value}")

('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00012_e') → Growth: 2.901984972733107e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00013_e') → Growth: -4.2705725209512465e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00030_e') → Growth: -8.85781508224639e-15
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00034_e') → Growth: -8.85781508224639e-15
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00048_e') → Growth: -1.88512474827295e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00058_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00063_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00067_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00099_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00149_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00205_e') → Growth: 1.1780889712968957e-14
('EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00254_e') → Growth: 1.1780

Ribose-5-P is missing in the model

In [ ]:
from cobra import Reaction, Metabolite

new_medium = {}

# Step 1: Create metabolite if not in model
if "cpd00050_e" not in model.metabolites:
    ribose5p = Metabolite(
        "cpd00050_e",
        formula="C5H9O8P",
        name="D-ribose 5-phosphate",
        compartment="e",
        charge=-2,
    )
    model.add_metabolites([ribose5p])
    print("Metabolite added")

# Step 2: Create the exchange reaction
if "EX_cpd00050_e" not in model.reactions:
    ex_ribose5p = Reaction("EX_cpd00050_e")
    ex_ribose5p.name = "Exchange of D-ribose 5-phosphate"
    ex_ribose5p.lower_bound = -10.0  # Allow uptake
    ex_ribose5p.upper_bound = 1000.0  # Maximum flux
    ex_ribose5p.add_metabolites({ribose5p: -1})
    model.add_reactions([ex_ribose5p])
    print("Exchange reaction added")

# Step 3: Refresh exchange reactions list 🔑
exchange_reactions = [rxn.id for rxn in model.exchanges]

# Step 4: Update the medium
baseline_medium = {
    'EX_cpd00099_e': 64.8,  # Chloride
    'EX_cpd00971_e': 91.6,  # Sodium
    'EX_cpd00048_e': 121.0,  # Sulfate
    'EX_cpd00205_e': 136.3,  # Potassium
    'EX_cpd00063_e': 0.2,  # Calcium
    'EX_cpd00149_e': 0.1,  # Carbonate
    'EX_cpd00254_e': 121.6,  # Magnesium
    'EX_cpd00029_e': 82.0,  # Acetate
    'EX_cpd00013_e': 53.5,  # Ammonium
    'EX_cpd00012_e': 136.1,  # H2PO4
    'EX_cpd00305_e': 0.014,  # Thiamine/Vit B1
    'EX_cpd00635_e': 0.0007378,  # Vit B12
    'EX_cpd00028_e': 0.010790591,  # Iron
    'EX_cpd00050_e': 10.0  # Ribose 5-phosphate
}

for nutrient, concentration in baseline_medium.items():
    if nutrient in exchange_reactions:
        new_medium[nutrient] = concentration
    else:
        print(f"Warning: {nutrient} not found in model exchange reactions")

# Step 5: Set the medium and optimize
model.medium = new_medium
solution = model.optimize()

print("Expected Growth Rate:", solution.objective_value)


Expected growth for our Baseline Media: 2.170149409370941e-15
